# Lab 2.4: LLM Observability with Arize Phoenix

In this lab, we will learn how to instrument our LLM applications for observability using **Arize Phoenix**. Observability is crucial for understanding how your LLM applications are performing, debugging issues, and evaluating quality.

We will cover:
1.  **Setup**: Installing Phoenix and launching the local UI.
2.  **Part 1: Simple LangChain Flow**: Instrumenting a basic chain.
3.  **Part 2: Advanced LangGraph Application**: Instrumenting a stateful agent workflow.

### Prerequisites
- Ensure you have your Groq API Key ready.


In [1]:
# 1. Install Dependencies
%pip install -qU arize-phoenix arize-otel openinference-instrumentation-langchain langchain langchain-groq langgraph
%pip install -q "arize[AutoEmbeddings]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.4/157.4 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.8/161.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.2/490.2 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# 2. Setup API Keys
import getpass
import os
from arize.otel import register

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

if "ARIZE_SPACE_ID" not in os.environ:
    os.environ["ARIZE_SPACE_ID"] = getpass.getpass("Enter your Arize Space ID: ")

if "ARIZE_API_KEY" not in os.environ:
    os.environ["ARIZE_API_KEY"] = getpass.getpass("Enter your Arize API Key: ")


Enter your Groq API Key: ··········
Enter your Arize Space ID: ··········
Enter your Arize API Key: ··········


## 3. Launch Arize Phoenix (Optional)
You can still launch the Phoenix application locally to view traces locally, or you can view them in the Arize Cloud dashboard.

In [3]:
# Import Arize observability components
from arize.otel import register
from getpass import getpass

# Configure Arize tracing
tracer_provider = register(
    space_id=os.environ["ARIZE_SPACE_ID"],
    api_key=os.environ["ARIZE_API_KEY"],
    project_name="arize-lab-demo"
)

# Enable automatic LangChain instrumentation
from openinference.instrumentation.langchain import LangChainInstrumentor
LangChainInstrumentor().instrument(tracer_provider=tracer_provider)

print("🔭 Arize observability configured successfully!")
print("📊 All LangChain operations will now be automatically traced")

🔭 OpenTelemetry Tracing Details 🔭
|  Arize Project: arize-lab-demo
|  Span Processor: BatchSpanProcessor
|  Collector Endpoint: otlp.arize.com
|  Transport: gRPC
|  Transport Headers: {'authorization': '****', 'api_key': '****', 'arize-space-id': '****', 'space_id': '****', 'arize-interface': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.

🔭 Arize observability configured successfully!
📊 All LangChain operations will now be automatically traced


## Part 1: Simple LangChain Flow (Banking Policy Assistant)

We will create a simple **Banking Policy Assistant** that answers internal policy questions. This simulates a basic RAG or Chatbot interaction.

In [4]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Setup a Banking Policy Assistant
llm = ChatGroq(
    model="qwen/qwen3-32b",
    temperature=0,
    reasoning_format="parsed"
)

# This prompt simulates an internal knowledge base lookup
policy_prompt = ChatPromptTemplate.from_template(
    "You are a Banking Policy Assistant. Answer the question based on standard mortgage guidelines. Keep it professional. Question: {question}"
)
chain = policy_prompt | llm | StrOutputParser()

# Invoke the chain
print("Invoking Banking Policy Assistant...")
question = "What is the maximum loan-to-value (LTV) ratio for a jumbo mortgage?"
print(f"Question: {question}")
response = chain.invoke({"question": question})
print(f"Response: {response}")

# Check the Phoenix UI and Arize Cloud to see the trace!

Invoking Banking Policy Assistant...
Question: What is the maximum loan-to-value (LTV) ratio for a jumbo mortgage?
Response: The maximum loan-to-value (LTV) ratio for a jumbo mortgage typically ranges between **80% and 90%**, depending on the lender and the borrower's financial profile. Most standard jumbo loan programs require a minimum down payment of **10% to 20%** (i.e., an LTV of 90% to 80%) to mitigate the higher risk associated with these larger-than-conforming loans. 

However, some lenders may offer higher LTV ratios (e.g., up to 95% or 97%) for borrowers with exceptional credit scores, stable income, and low debt-to-income ratios. These exceptions often come with stricter underwriting criteria and may require private mortgage insurance (PMI) or other risk-mitigation measures. Always confirm specific requirements with the lender, as policies can vary.


## Part 2: Advanced LangGraph Application (Loan Underwriting)

Now, let's look at a complex **Loan Underwriting Pipeline**. This involves multiple steps: Risk Analysis and Final Decision.

We will use LangGraph to model this stateful workflow, where a **Risk Analyst** assesses the applicant and a **Senior Underwriter** makes the final call.

### 2.1 Define State
We define a complex state capable of holding the applicant profile and the calculated risk score.

In [5]:
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

# --- 2. STATE DEFINITION ---
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    applicant_id: str
    risk_score: str

### 2.2 Define Risk Analyst Node
This node analyzes the financial data and assigns a risk level.

In [6]:
# Define Nodes
def risk_analyst_node(state: State):
    """Analyzes the financial health of the applicant."""
    print("--- Node: Risk Analyst ---")
    applicant_data = state["messages"][-1].content

    system_msg = (
        "You are a Risk Analyst for a bank. Analyze the provided applicant financial data "
        "(Income, Debt, Credit Score). Calculate the Debt-to-Income (DTI) ratio. "
        "Summarize the financial health and assign a risk level (Low, Medium, High)."
    )

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_msg),
        ("human", "Applicant Data: {data}")
    ])
    chain = prompt | llm
    response = chain.invoke({"data": applicant_data})

    # Extract a dummy risk score to update state (simplification)
    # In a real app, we'd use structured output
    return {"messages": [response], "risk_score": "Calculated"}

### 2.3 Verification: Test Risk Analyst
Let's ensure the risk analyst can corectly interpret data.

In [7]:
# Verification
print("Testing Risk Analyst Node...")
sample_data = "Income: $100k, Debt: $2k/mo, Credit: 700"
state = {"messages": [HumanMessage(content=sample_data)]}
result = risk_analyst_node(state)
print("Analysis:", result['messages'][0].content)

Testing Risk Analyst Node...
--- Node: Risk Analyst ---
Analysis: **Analysis Summary:**  
- **Debt-to-Income (DTI) Ratio:**  
  Monthly Debt: $2,000 | Monthly Income: $100,000 ÷ 12 = **~$8,333**  
  DTI = ($2,000 ÷ $8,333) × 100 = **~24%**  

- **Financial Health:**  
  - **DTI Interpretation:** 24% falls within the **acceptable range** (20–36%), indicating manageable debt relative to income.  
  - **Credit Score:** 700 is a **good score** (700–749 is considered "good" in most credit scoring models).  

**Risk Level:** **Low**  
The applicant demonstrates strong financial health with a low DTI and a solid credit score, suggesting a low risk of default.


### 2.4 Define Senior Underwriter Node
This node takes the risk analysis and makes the final decision.

In [8]:
def senior_underwriter_node(state: State):
    """Makes the final loan decision based on risk analysis."""
    print("--- Node: Senior Underwriter ---")
    risk_analysis = state["messages"][-1].content

    system_msg = (
        "You are a Senior Underwriter. Review the Risk Analyst's report. "
        "Make a final decision: APPROVE or DENY. "
        "If approved, set the interest rate tier (Tier 1 is best). "
        "Provide a brief justification for the decision."
    )

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_msg),
        ("human", "Risk Analysis Report: {report}")
    ])
    chain = prompt | llm
    response = chain.invoke({"report": risk_analysis})
    return {"messages": [response]}

### 2.5 Build and Compile Graph
Connect the nodes to form the pipeline.

In [9]:
# Build Graph
workflow = StateGraph(State)

# Add Nodes
workflow.add_node("risk_analyst", risk_analyst_node)
workflow.add_node("senior_underwriter", senior_underwriter_node)

# Define Logic
workflow.add_edge(START, "risk_analyst")
workflow.add_edge("risk_analyst", "senior_underwriter")
workflow.add_edge("senior_underwriter", END)

# Compile
graph = workflow.compile()

### 2.6 Run the Pipeline
Now we run the full flow and can observe the trace in Arize Phoenix.

In [10]:
# Invoke Graph
print("🚀 Starting Loan Underwriting Pipeline...")

applicant_info = "Applicant: John Doe. Income: $150,000/yr. Monthly Debt: $2,500. Credit Score: 780. Loan Amount: $500,000."
print(f"Processing Application: {applicant_info}\n")

inputs = {
    "messages": [HumanMessage(content=applicant_info)],
    "applicant_id": "APP-12345"
}

for output in graph.stream(inputs, stream_mode="updates"):
    for node_name, node_state in output.items():
        print(f"\n--- NODE: {node_name.upper()} ---")
        last_msg = node_state["messages"][-1]
        print(f"Content: {last_msg.content[:300]}...")

print("\n✅ Pipeline Complete. Check Phoenix UI for the trace lineage.")

# Check Phoenix UI again. You should see a trace for this execution showing the graph steps.

🚀 Starting Loan Underwriting Pipeline...
Processing Application: Applicant: John Doe. Income: $150,000/yr. Monthly Debt: $2,500. Credit Score: 780. Loan Amount: $500,000.

--- Node: Risk Analyst ---

--- NODE: RISK_ANALYST ---
Content: **Financial Analysis for John Doe**  

### **Key Calculations**  
1. **Debt-to-Income (DTI) Ratio**:  
   - **Monthly Gross Income**: $150,000 / 12 = **$12,500**  
   - **DTI**: ($2,500 Monthly Debt / $12,500 Monthly Income) × 100 = **20%**  

2. **Loan-to-Income Ratio**:  
   - **Loan Amount / Annu...
--- Node: Senior Underwriter ---

--- NODE: SENIOR_UNDERWRITER ---
Content: **Decision**: APPROVE  
**Interest Rate Tier**: Tier 1  

**Justification**:  
John Doe exhibits a **low-risk profile** with a 20% DTI, excellent credit score (780), and a strong income ($150,000 annually). While the loan-to-income ratio is high (3.33x), his robust financial capacity and disciplined...

✅ Pipeline Complete. Check Phoenix UI for the trace lineage.


## Part 3: High-Level Chaining with Runnables (Customer Feedback Processor)

In this section, we will see how Arize Phoenix shines when debugging complex chains using `RunnablePassthrough` and `RunnableLambda`.
We often want to pass data through multiple steps, augmenting it along the way.

We will build a **Customer Feedback Processor** that:
1.  Takes a raw review.
2.  **Cleans** it (custom function).
3.  **Analyzes Sentiment** (LLM call).
4.  **Extracts Keywords** (LLM call).
5.  **Generates a Summary Response** using all previous inputs.

In [11]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# 1. Define specific purpose chains
sentiment_prompt = ChatPromptTemplate.from_template(
    "Analyze the sentiment of this review. Return ONLY one word: POSITIVE, NEGATIVE, or NEUTRAL. Review: {cleaned_review}"
)
sentiment_chain = sentiment_prompt | llm | StrOutputParser()

keyword_prompt = ChatPromptTemplate.from_template(
    "Extract top 3 keywords from this review. Return them as a comma-separated list. Review: {cleaned_review}"
)
keyword_chain = keyword_prompt | llm | StrOutputParser()

final_response_prompt = ChatPromptTemplate.from_template(
    "You are a Customer Success Manager. Write a generic response to this review based on the analysis.\n"
    "Review: {cleaned_review}\n"
    "Sentiment: {sentiment}\n"
    "Keywords: {keywords}\n\n"
    "Response:"
)
final_response_chain = final_response_prompt | llm | StrOutputParser()

# 2. Define custom helper (RunnableLambda)
def clean_text(text):
    return text.strip().lower()

# 3. Build the Super Chain with Passthrough
# The input to the chain is just the raw key "review"
super_chain = (
    {"cleaned_review": lambda x: clean_text(x["review"])}
    | RunnablePassthrough.assign(sentiment=sentiment_chain)
    | RunnablePassthrough.assign(keywords=keyword_chain)
    | RunnablePassthrough.assign(display_result=final_response_chain)
)

# Invoke
print("🚀 Processing Customer Feedback...")
review_text = "  The product arrived late and was broken. Terrible service!  "
result = super_chain.invoke({"review": review_text})

print(f"\nOriginal: '{review_text}'")
print(f"Cleaned: '{result['cleaned_review']}'")
print(f"Sentiment: {result['sentiment']}")
print(f"Keywords: {result['keywords']}")
print(f"\nFinal Response generated:\n{result['display_result']}")

# Check Arize Phoenix! You will see the 'sentiment_chain' and 'keyword_chain'
# as separate spans running in parallel (conceptually) or sequentially,
# feeding into the final step.

🚀 Processing Customer Feedback...

Original: '  The product arrived late and was broken. Terrible service!  '
Cleaned: 'the product arrived late and was broken. terrible service!'
Sentiment: NEGATIVE
Keywords: late, broken, terrible service

Final Response generated:
**Response:**  
Dear [Customer Name],  

Thank you for taking the time to share your feedback. We sincerely apologize for the inconvenience you experienced with your order—both the delayed delivery and the damaged product are unacceptable, and we deeply regret the frustration this caused.  

To resolve this, we will immediately dispatch a replacement at no cost to you and expedite the shipping to ensure it arrives promptly. Additionally, we’d like to investigate the shipping process to identify where things went wrong and prevent future occurrences.  

Your satisfaction is our priority, and we’re committed to improving our service. Please reply to this email with your order number so we can escalate your case and provide a

## Conclusion
You have successfully set up Arize Phoenix and instrumented both a **Banking Policy Assistant** and a **Loan Underwriting Pipeline**.
Review the traces in the Phoenix UI to understand the internal execution of your Critical Financial Workflows.